In [1]:
import pandas as pd
import numpy as np
import datetime
import ast
import warnings
from ast import literal_eval
import string
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import GridSearchCV, KFold, train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
warnings.filterwarnings('ignore')
import pickle
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds
from sklearn.preprocessing import StandardScaler
from scipy.sparse import coo_matrix, csr_matrix, save_npz, load_npz, hstack
import random
from sklearn.metrics import mean_squared_error

favorites = ['The Count of Monte Cristo (2002)',"Lord of the Rings: The Return of the King, The (2003)",'The Imitation Game (2014)']
non_favorites = ['Around the World in 80 Days (2004)', 'Gentlemen of Fortune (Dzhentlmeny udachi) (1972)'
                 , 'Friday the 13th Part 3: 3D (1982)', 'Swing Vote (2008)', 'Fish Called Wanda, A (1988)'
                 , 'Dark Blue (2003)', 'We Bought a Zoo (2011)', 'Mouchette (1967)', 'My Favorite Wife (1940)'
                 , '24 Hour Party People (2002)', 'American Me (1992)', 'Your Friends and Neighbors (1998)'
                 , 'Amarcord (1973)', 'Bob le Flambeur (1955)', 'Songs From the Second Floor (Sånger från andra våningen) (2000)'
                 , 'Bliss (1997)', 'Night of the Creeps (1986)', 'Best Exotic Marigold Hotel, The (2011)', 'Pandorum (2009)'
                ,"Cheech & Chong's The Corsican Brothers (1984)", 'Great Buck Howard, The (2008)', 'Hamburger Hill (1987)', 
                 'Head of State (2003)', 'Morvern Callar (2002)', 'Harvard Man (2001)', "Relax... It's Just Sex (1998)"
                 , 'Zeus and Roxanne (1997)', 'Poseidon Adventure, The (1972)', 'Here Comes the Boom (2012)']

In [2]:
moviesdf = pd.read_csv(r'C:\Users\marsh\Desktop\Data_Science_Projects_Marshall_lee\Movie_Recommender\website\moviesdf4.csv')
ratingsdf = pd.read_csv('ratingsdf3.csv')

In [3]:
pivot = ratingsdf.pivot(index = 'title',columns = 'userId', values = 'rating')
pivot

userId,1,2,3,5,7,8,11,13,14,16,...,138474,138475,138477,138483,138484,138486,138487,138490,138492,138493
title,,,,,,,,,,,,,,,,,,,,,
"""Great Performances"" Cats (1998)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Round Midnight (1986),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Salem's Lot (2004),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Til There Was You (1997),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"'burbs, The (1989)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
eXistenZ (1999),NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
xXx (2002),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
xXx: State of the Union (2005),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
train_data, test_data = train_test_split(pivot, test_size = 0.2)
scaler = StandardScaler()
train_data_scaled = scaler.fit_transform(train_data)
test_data_scaled = scaler.transform(test_data)
test_data_scaled = np.nan_to_num(test_data_scaled, nan = 0)
train_data_scaled = np.nan_to_num(train_data_scaled, nan = 0)
num_latent_factors = 10
U, s, Vt = svds(train_data_scaled, k=num_latent_factors)
s_diag = np.diag(s)
predicted_ratings = np.dot(np.dot(U, s_diag), Vt)

# Evaluate the model on the test set
nonzero_indices = test_data_scaled.nonzero()
test_ratings = test_data_scaled[nonzero_indices]
test_row_indices = nonzero_indices[0]
test_col_indices = nonzero_indices[1]
predicted_test_ratings = predicted_ratings[test_row_indices, test_col_indices]
rmse = np.sqrt(mean_squared_error(test_ratings, predicted_test_ratings))
mse = mean_squared_error(test_ratings, predicted_test_ratings)
print(f"RMSE: {rmse:.3f}, MSE: {mse:.3f}")

In [4]:
scaler = StandardScaler()
scaler = scaler.fit_transform(pivot)
ui_std = np.nan_to_num(scaler, nan=0)
ui_std
left_matrix = ui_std[:, :41851]
right_matrix = ui_std[:, 41851:]
left_matrix_sparse = csr_matrix(left_matrix)
right_matrix_sparse = csr_matrix(right_matrix)

In [8]:
left_matrix.shape, right_matrix.shape

((9143, 41851), (9143, 41850))

In [11]:
#save_npz('right_matrix_sparse.npz',left_matrix_sparse)

In [12]:
#save_npz('left_matrix_sparse.npz',right_matrix_sparse)

In [13]:
# User item matrix with standardscaler (by user)

#save_npz('user_item_matrix.npz',sparse_matrix)
right = csr_matrix(load_npz(r'C:\Users\marsh\Desktop\Data_Science_Projects_Marshall_lee\Movie_Recommender\website\right_matrix_sparse.npz'))
left = csr_matrix(load_npz(r'C:\Users\marsh\Desktop\Data_Science_Projects_Marshall_lee\Movie_Recommender\website\left_matrix_sparse.npz'))

In [20]:
sparsity = 1.0 -( ui_new.count_nonzero() / float(ui_new.toarray().size) )
sparsity

0.9766667942962802

In [17]:
def recommended(favorites,non_favorites):
    newdf = pd.DataFrame(index = moviesdf.title)
    newdf['new_user_rating'] = np.nan
    newdf.loc[favorites,'new_user_rating'] = 5
    newdf.loc[non_favorites,'new_user_rating'] = 0.5
    scaler = StandardScaler()
    scaled = scaler.fit_transform(newdf)
    scaled = np.nan_to_num(scaled, nan=0)
    sparse_scaled = csr_matrix(scaled)
    ui_new = hstack([left, right, sparse_scaled])
    U,S, Vt = svds(ui_new, k=10)
    Uk = U
    Sk = np.diag(S)
    Vk = Vt.T
    # Compute the predicted ratings for the new user using the truncated SVD
    new_user_ratings = Uk @ Sk @ Vk[-1,:].T
    rec = pd.DataFrame(data= new_user_ratings, columns = ['rating'])
    rec['title'] = moviesdf.title
    rec = rec[~rec.title.isin(favorites)]
    rec = rec.sort_values('rating', ascending = False).head(10)
    rec = rec.merge(moviesdf, on = 'title',how ='inner')
    return rec

In [18]:
recommended_movies = recommended(favorites,non_favorites)
recommended_movies.title

0    Lord of the Rings: The Fellowship of the Ring,...
1        Lord of the Rings: The Two Towers, The (2002)
2                                   Matrix, The (1999)
3    Pirates of the Caribbean: The Curse of the Bla...
4                                     Gladiator (2000)
5                              Dark Knight, The (2008)
6                                         Shrek (2001)
7                                  Finding Nemo (2003)
8                                 Batman Begins (2005)
9                              Incredibles, The (2004)
Name: title, dtype: object

In [ ]:
newdf = pd.DataFrame(index = moviesdf.title)
newdf['new_user_rating'] = np.nan
newdf.loc[favorites,'new_user_rating'] = 5
newdf.loc[non_favorites,'new_user_rating'] = 0.5
scaler = StandardScaler()
scaled = scaler.fit_transform(newdf)
scaled = np.nan_to_num(scaled, nan=0)
sparse_scaled = csr_matrix(scaled)


In [ ]:
#from sklearn.base import BaseEstimator

#class SVDSEstimator(BaseEstimator):
#    def __init__(self, k=50):
#        self.k = k

#    def fit(self, X, y=None):
#        U, S, Vt = svds(X, k=self.k)
#        self.Uk = U
#        self.Sk = np.diag(S)
#        self.Vk = Vt.T
#        return self

#    def transform(self, X):
#        return self.Uk @ self.Sk @ self.Vk.T
    
#estimator = SVDSEstimator()
#param_grid = {'k': [10,50,100,150,200]}
#grid_search = GridSearchCV(estimator=estimator, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5)
#grid_search.fit(ui)

In [ ]:
#best_k = grid_search.best_params_['k']

#best_k